In [1]:
import numpy as np
import pandas as pd
import uproot
from pylorentz import Momentum4
import matplotlib.pyplot as plt
%matplotlib inline
m_tau = 1.776

In [2]:
df = pd.read_pickle('../shared/df_tt_rho_rho.pkl')

In [6]:
df = df[(df['mva_dm_1'] == 1) & (df['mva_dm_2'] == 1) & (df["tau_decay_mode_1"] == 1) & (df["tau_decay_mode_2"] == 1)]

In [7]:
df.head()

,wt_cp_sm,wt_cp_ps,wt_cp_mm,rand,aco_angle_1,mva_dm_1,mva_dm_2,tau_decay_mode_1,tau_decay_mode_2,pi_E_1,...,metcov00,metcov01,metcov10,metcov11,gen_nu_p_1,gen_nu_phi_1,gen_nu_eta_1,gen_nu_p_2,gen_nu_phi_2,gen_nu_eta_2
entry,,,,,,,,,,,,,,,,,,,,,
8,1.228852,0.950417,0.223672,0.861532,2.657029,1,1,1,1,45.423448,...,42.660179,48.254131,48.254131,127.915657,11.907320,-1.878383,-0.443223,12.628682,1.261835,0.048225
25,0.124674,1.936855,1.318387,0.932849,1.547246,1,1,1,1,24.506373,...,161.991516,2.567861,2.567861,105.209160,14.349887,2.519850,-0.934011,44.844142,-0.313698,-1.423014
27,1.461517,0.400455,0.172710,0.132842,3.750274,1,1,1,1,15.319610,...,80.564423,-29.174086,-29.174086,72.426666,35.673293,0.106444,1.180579,23.342057,-2.873663,1.126849
45,0.059870,0.061072,0.111744,0.514073,1.192612,1,1,1,1,94.211361,...,45.858810,-4.467490,-4.467490,212.978531,2.964097,-0.686183,-1.008616,0.729467,2.953007,-0.809572
50,0.654131,0.931771,0.072470,0.504356,3.264792,1,1,1,1,25.899289,...,84.956879,27.219664,27.219664,33.637897,8.663849,-2.653995,-0.131161,27.029359,0.369547,-0.154201


In [16]:
pi_1 = Momentum4(df['pi_E_1'], df['pi_px_1'], df['pi_py_1'], df['pi_pz_1'])
pi_2 = Momentum4(df['pi_E_2'], df['pi_px_2'], df['pi_py_2'], df['pi_pz_2'])
pi0_1 = Momentum4(df['pi0_E_1'], df['pi0_px_1'], df['pi0_py_1'], df['pi0_pz_1'])
pi0_2 = Momentum4(df['pi0_E_2'], df['pi0_px_2'], df['pi0_py_2'], df['pi0_pz_2'])

In [96]:
p1 = pi_1
p2 = pi_2
p3 = pi0_1
p4 = pi0_2
rest_frame = pi_1+pi_2+pi0_1+pi0_2
boost = Momentum4(rest_frame[0], -rest_frame[1], -rest_frame[2], -rest_frame[3])
p1 = p1.boost_particle(boost)
p2 = p2.boost_particle(boost)
p3 = p3.boost_particle(boost)
p4 = p4.boost_particle(boost)

# Some geometrical functions
def cross_product(vector3_1, vector3_2):
    return np.cross(vector3_1.T, vector3_2.T).T

def dot_product(vector1, vector2):
    return np.einsum('ij, ij->i', vector1.T, vector2.T)

def norm(vector):
    return np.sqrt((vector.T ** 2).sum(-1))[..., np.newaxis].T

# calculating the perpependicular component
pi0_1_3Mom_star_perp = cross_product(p1[1:], p3[1:])
pi0_2_3Mom_star_perp = cross_product(p2[1:], p4[1:])
# Now normalise:
pi0_1_3Mom_star_perp = pi0_1_3Mom_star_perp/norm(pi0_1_3Mom_star_perp)
pi0_2_3Mom_star_perp = pi0_2_3Mom_star_perp/norm(pi0_2_3Mom_star_perp)
# Calculating phi_star
phi_CP = np.arccos(dot_product(pi0_1_3Mom_star_perp, pi0_2_3Mom_star_perp))

In [109]:
phi_CP

array([0.48437372, 1.59424942, 0.60866791, ..., 1.28933097, 2.63131196,
       1.21207559])

In [121]:
p1 = pi_1
p2 = pi_2
p3 = pi0_1
p4 = pi0_2
rest_frame = pi_1+pi_2+pi0_1+pi0_2

boost = Momentum4(rest_frame[0], -rest_frame[1], -rest_frame[2], -rest_frame[3])
p1_boosted = p1.boost_particle(boost)
p2_boosted = p2.boost_particle(boost)
p3_boosted = p3.boost_particle(boost)
p4_boosted = p4.boost_particle(boost)
p1_b_p = np.c_[p1_boosted.p_x, p1_boosted.p_y, p1_boosted.p_z]
p2_b_p = np.c_[p2_boosted.p_x, p2_boosted.p_y, p2_boosted.p_z]
p3_b_p = np.c_[p3_boosted.p_x, p3_boosted.p_y, p3_boosted.p_z]
p4_b_p = np.c_[p4_boosted.p_x, p4_boosted.p_y, p4_boosted.p_z]
n1 = p1_b_p - np.multiply(np.einsum('ij, ij->i', p1_b_p, normaliseVector(p3_b_p))[:, None], normaliseVector(p3_b_p))
n2 = p2_b_p - np.multiply(np.einsum('ij, ij->i', p2_b_p, normaliseVector(p4_b_p))[:, None], normaliseVector(p4_b_p))
n1 = normaliseVector(n1)
n2 = normaliseVector(n2)
# vectorised form of
# n1 = p1.Vect() - p1.Vect().Dot(p3.Vect().Unit())*p3.Vect().Unit();
# n2 = p2.Vect() - p2.Vect().Dot(p4.Vect().Unit())*p4.Vect().Unit();
aco_angle = np.arccos(np.einsum('ij, ij->i', n1, n2))

def normaliseVector(vec):
    """

    Normalises an array of vectors
    """
    return vec/np.sqrt((vec ** 2).sum(-1))[..., np.newaxis]

def dotRowOfVectors(vec1, vec2):
    return np.einsum('ij, ij->i', vec1, vec2)

In [120]:
np.array(aco_angle)

array([3.61897216, 4.6924002 , 3.72700124, ..., 4.40393606, 5.72403402,
       4.32747216])

In [115]:
np.einsum('ij, ij->i', p1_b_p, normaliseVector(p3_b_p))

array([38.78627605, 17.15501438,  8.51852755, ..., 48.04197876,
        1.96696487, 14.98627838])

In [52]:
np.einsum('ij, ij->i', p1_b_p, normaliseVector(p3_b_p))[:, None]

array([[ 8.55104912],
       [29.05548573],
       [37.54481643],
       ...,
       [ 2.68961678],
       [33.24603664],
       [13.17196788]])

In [122]:
y_1 = (pi_1_boosted.E - pi0_1_boosted.E)/(pi_1_boosted.E + pi0_1_boosted.E)
y_2 = (pi_2_boosted.E - pi0_2_boosted.E)/(pi_2_boosted.E + pi0_2_boosted.E)

NameError: name 'pi_1_boosted' is not defined